# **Character Level Multi Layer Perceptron Language Model**
This is inspired by [A Neural Probabilistic Language Model](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) paper.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
# load dataset
words = open('names.txt', 'r').read().splitlines()

In [ ]:
# create lookup table for converting characters to indices
chars = sorted(list(set(''.join(words)))) # all unique characters in the dataset
stoi = {s:i+1 for i,s in enumerate(chars)} # string to index

# manually enumerate start and end token since they are not visible in the dataset
start_token = '<'
end_token = '>'
stoi[start_token] = 0
stoi[end_token] = len(stoi)

# total number of unique characters plus start and end token
chars_count = len(stoi)

# index to string
itos = {i:s for s,i in stoi.items()}

itos

### Build dataset
Based on given list of words creates input tensor with sequence of characters  with length equal to 'context_length' and target tensor with next character in sequence.

In [ ]:
def build_dataset(words, context_length=3):
    X, Y = [], []
    for w in words:
        context = [stoi[start_token]] * context_length # context window padded with start token
        for ch in w + end_token:
            ix = stoi[ch]
            X.append(context) # for given context ...
            Y.append(ix) # ... the next character is the target
            context = context[1:] + [ix] # crop and append - sliding window
    
    return torch.tensor(X), torch.tensor(Y)

In [ ]:
# X (input) and Y (output/target/label) tensors
X, Y = build_dataset(words[:1])
print(f'X.shape = {X.shape}, X.dtype = {X.dtype}')
print(f'Y.shape = {Y.shape}, Y.dtype = {Y.dtype}')

print(f'For the first word in the dataset: {start_token + words[0] + end_token}')
for i in range(len(X)):
    print(f"X[{i}] = {X[i].tolist()} = {''.join([itos[j] for j in X[i].tolist()])}"
          f" ---and target char is--> Y[{i}] = {Y[i]} = {itos[Y[i].item()]}")

### Create embedding lookup table

In [ ]:
# Matrix of embeddings
C = torch.randn((chars_count, 2)) # 2-dimensional embeddings

# Example of embedding single character with index 15
# since our matrix of embeddings is the same size as the number of characters
# we can simply use the index of the character to get its embedding
example_embedding = C[15]
print(f'Embedding using index C[15]={example_embedding}')

# Alternative approach would be to one-hot encode the character and then multiply it by the embedding matrix
# this will give same result because encoded vector will have only one non-zero value equal to 1
# and this will simply act as a mask for the embedding matrix
example_embedding = F.one_hot(torch.tensor([15]), num_classes=chars_count).float() @ C
print(f'Embedding using one-hot encoding and matrix multiplication={example_embedding}')

# For rest of of notebook I will use index based approach because it is more efficient
# also thanks to python semantics we can easly retrieve embedding for whole sequence of characters
# by indexing with list or tensor of integers
example_embedding = C[torch.tensor([15, 20, 5])]
print(f'Embedding for sequence of three characters C[torch.tensor([15, 20, 5])]=\n{example_embedding}')


In [ ]:
# What is even more mind blowing is that we can index using multidimensional tensor!
# in this exaple where X is 5x3 tensor, we will get 5x3x2 tensor with embeddings for each character in each sequence in third dimension
print(f'Shape of C[X].shape={C[X].shape}')
print(f'And C[X] =\n{C[X]}')
print(f'We can also access individual ebeddings for given sequence, for example C[X][0] =\n{C[X][0]}')
print(f'Or even embedding for individual character in sequence, for example C[X][0][0] =\n{C[X][0][0]}')

In [ ]:
# Embeddings for imput sequences
X_embedded = C[X]

### MLP input and hidden layers

In [ ]:
# Number of inputs for first layer
# context length * number of embeddings for each character
# in our case 3 * 2 = 6 which is the same as shape[1] * shape[2] of X_embedded
# and we will pass embeddings for each character in the sequence as input to the network
input_layer_size = X_embedded.shape[1] * X_embedded.shape[2]
print(f'input_layer_size={input_layer_size}')

# number of neurons in hidden layer
hidden_layer_size = 100
print(f'hidden_layer_size={hidden_layer_size}')

# W1 represents neural network weights between input and hidden layer
W1 = torch.randn((input_layer_size, hidden_layer_size))
print(f'W1.shape={W1.shape}')

# b1 represents bias for hidden layer
b1 = torch.randn(hidden_layer_size)
print(f'b1.shape={b1.shape}')

In [ ]:
# In order to perform matrix multiplication between input and weights we need to 
# transform input tensor which is 5x3x2 into 5x6 to fit into neural network

# This is how to access all embeddings for first character in each sequence
print(f'X_embedded[:, 0, :]=\n{X_embedded[:, 0, :]}\n')

# Knowing above we can concatenate all embeddings for each character in each sequence
print(f'torch.concat + tensor list=\n{torch.concat([X_embedded[:, i, :] for i in range(X_embedded.shape[1])], dim=1)}\n')

# We can also improve it by using torch.unbind which will return list of tensors and yield same result as for loop
print(f'torch.concat + torch.unbind=\n{torch.concat(torch.unbind(X_embedded, dim=1), dim=1)}\n')

In [ ]:
# But it turns out there is another great function in pytorch called torch.view
# Lets have some fun with it

a = torch.arange(16) # 1D tensor with 16 elements
print(f'a.shape={a.shape}, a={a}\n')
print(f'a.view(2, 8)=\n{a.view(2, 8)}\n')
print(f'a.view(4, 4)=\n{a.view(4, 4)}\n')
print(f'a.view(2, 2, 4)=\n{a.view(2, 2, 4)}\n')

In [ ]:
# Knowing above we can use torch.view to transform our 5x3x2 tensor into 5x6
# Basically we just keep first dimension and merge all other dimensions into one
# Also it looks like 6 is the only possible value for second dimension because 3*2=6
# Knowing that we can use -1 for second dimension and pytorch will figure out the rest
X_embedded_flat = X_embedded.view(-1, 6)
print(f'X_embedded_flat.shape={X_embedded_flat.shape}\n')
print(f'X_embedded_flat=\n{X_embedded_flat}\n')

### First layer multiplication to get outputs from hidden layer
Our current desing of network is: 6 inputs which represents 2 dimensional embeddings for 3 character long sequence that go into 100 neurons in hidden layer and what we get is 100 outputs for each input sequence..

In [ ]:
h = torch.tanh(X_embedded_flat @ W1 + b1) 
# + b1 is broadcasted to match shape of X_embedded_flat @ W1
# 5, 100
# 1, 100 - broadcasting will align on the right, create fake dimension on the left
# and will be copy b1 values 5 times to match shape of (X_embedded_flat @ W1) and perform elementwise addition
# and will result of adding 5x100 matrix to 5x100 matrix with copied values from b1
print(f'h.shape={h.shape}\n')

### Output layer
Takes 100 outputs from hidden layer and produces output that represent one_hot_encoding for our characters set for each input sequence.

In [ ]:
W2 = torch.randn((hidden_layer_size, chars_count))
b2 = torch.randn(chars_count)

In [ ]:
# forwad pass for output layer
logits = h @ W2 + b2

# calculate output probabilities (the old way, manualy calculating softmax)
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)

# but there is faster way to calculate probabilities - use torch.nn.functional.softmax
# and this is equivalent to above calculation
probs = F.softmax(logits, dim=1)
probs.shape

In [ ]:
# And what we see here is output character probabilities for second input sequence
probs[1]

### Network evaluation

In [ ]:
# iterator over input sequences
it = torch.arange(X_embedded.shape[0])

# extract probabilities for target characters
target_probs = probs[it, Y]

# calculate loss as average negative log likelihood
loss = -torch.log(target_probs).mean()

print(f'Loss before training = {loss}')

# Keep it short and clean without outputs
Above code show detail example of how MLP language model works with one input word.  Since I already went step by step with relevant outputs and example is clear next step is to make this code compact and efficient so it can be used for training and evaluation of language model for whole dataset.

In [ ]:
# Build dataset for training
X, Y = build_dataset(words, context_length=3)

# Create generator for reproducibility
g = torch.Generator().manual_seed(42)

# Create embedding lookup table with size chars_count
embedding_size = 2
C = torch.randn((chars_count, embedding_size), generator=g)

# Embed input sequences
X_embedded = C[X]

# Flatten embedded input sequences, to keep embeddings in one dimension
input_layer_size = X_embedded.shape[1] * X_embedded.shape[2] # context_length * embedding_size
X_embedded_flat = X_embedded.view(-1, input_layer_size)

# Create first neuron layer
hidden_layer_size = 100
W1 = torch.randn((input_layer_size, hidden_layer_size), requires_grad=True, generator=g)
b1 = torch.randn(hidden_layer_size, requires_grad=True, generator=g)

# Create second neuron layer
W2 = torch.randn((hidden_layer_size, chars_count), requires_grad=True, generator=g)
b2 = torch.randn(chars_count, requires_grad=True, generator=g)

# Parameters of the network
params = [C, W1, b1, W2, b2]
print(f'Number of parameters = {sum([p.numel() for p in params])}')

In [ ]:
def forward_pass():
    h = torch.tanh(X_embedded_flat @ W1 + b1)
    logits = h @ W2 + b2
    # Calculate mean negative log likelihood based on softmax made of logits
    # using torch.nn.functional.cross_entropy which is give same result as:
    # counts = logits.exp() - also this may cause trobles if logits are too big we will get nan
    # probs = counts / counts.sum(dim=1, keepdim=True)
    # loss = -torch.log(probs[range(len(Y)), Y]).mean()
    # ..but much meory efficient because we do not create intermediate tensors..
    # ..and much faster and also makes backpropagation easier :)
    return F.cross_entropy(logits, Y)

In [ ]:
def backward_pass(loss, weigths_and_biases):
    for p in weigths_and_biases:
        p.grad = None
    loss.backward()   